In [15]:
include("$(pwd())/instance_generation.jl")
using BilevelJuMP

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-03


In [16]:
TotalFacilities = 10
TotalClients = 30
TotalScenarios = 50

# Solving the full space problem for reference
instance = generate_instance(TotalFacilities, TotalClients, TotalScenarios);

In [17]:
function generate_full_problem_body(instance::Instance)
    
    I, J, S, N, P, O, V, U, T, D, bigM = unroll_instance(instance)

    # Initialize model
    m = Model(myGurobi)
    
    # Decision variables
    @variable(m, x[I], Bin)     # 1 if facility is located at i ∈ I, 0 otherwise.
    @variable(m, y[I] >= 0)     # Capacity decided for facility i ∈ I
    @variable(m, w[I,J,S] >= 0) # Flow between facility i ∈ I and client j ∈ J in scenario s ∈ S
    @variable(m, z[J,S] >= 0)   # Shortage in location j ∈ J in scenario s ∈ S

    # Constraints
    # Maximum number of servers
    @constraint(m, numServers,
        sum(x[i] for i in I) <= N
    )
    
    # Capacity limits: cannot deliver more than capacity decided, 
    #   and only if facility was located
    @constraint(m, capBal[i in I, s in S],
        sum(w[i,j,s] for j in J) <=  y[i]
    )

    @constraint(m, capLoc[i in I, s in S], 
        sum(w[i,j,s] for j in J) <= x[i] * bigM
    )
    
    # Demand balance: Demand of active clients must be fulfilled
    @constraint(m, demBal[j in J, s in S],
        sum(w[i,j,s] for i in I) >= D[j,s] - z[j,s]
    )
    
    return m  # Return the generated model
end

generate_full_problem_body (generic function with 1 method)

In [18]:
function generate_2SSP_model!(instance, m)
    I, J, S, N, P, O, V, U, T, D, bigM = unroll_instance(instance)
    x = m[:x]
    y = m[:y]
    w = m[:w]
    z = m[:z]

    # The two-stage objective function
    FirstStage = @expression(m, 
        sum(O[i] * x[i] + V[i] * y[i] for i in I) 
    )

    SecondStage = @expression(m, 
        sum(P[s] * (
                sum(T[i,j] * w[i,j,s] for i in I, j in J)
                + sum(U * z[j,s] for j in J)) 
        for s in S)
    )
    
    @objective(m, Min, FirstStage + SecondStage)

    return m
end

function generate_minimax!(instance, m)
    I, J, S, N, P, O, V, U, T, D, bigM = unroll_instance(instance)
    x = m[:x]
    y = m[:y]
    w = m[:w]
    z = m[:z]
    
    @variable(m, θ >= 0)

    FirstStage = @expression(m, 
        sum(O[i] * x[i] + V[i] * y[i] for i in I) 
    )

    @constraint(m, Minimax[s in S], 
        θ >= sum(T[i,j] * w[i,j,s] for i in I, j in J) + sum(U * z[j,s] for j in J)
    )   

    @objective(m, Min, FirstStage + θ)

    return m
end

generate_minimax! (generic function with 1 method)

In [19]:
model1 = generate_full_problem_body(instance)
model2 = generate_full_problem_body(instance)

# Solving 2SSP
expected_value_model = generate_2SSP_model!(instance, model1)
optimize!(expected_value_model)

# Solving the minimax
minimax_model = generate_minimax!(instance, model2)
optimize!(minimax_model)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 2501 rows, 16520 columns and 47510 nonzeros
Model fingerprint: 0xb5fd7256
Variable types: 16510 continuous, 10 integer (10 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e-02, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 7e+01]
Found heuristic solution: objective 423740.00000
Presolve time: 0.03s
Presolved: 2501 rows, 16520 columns, 47510 nonzeros
Variable types: 16510 continuous, 10 integer (10 binary)

Root relaxation: objective 4.483526e+03, 3212 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4483.52575    0    3 423740.000 4483.52575  98.9%     -    0s
     0     0 4483.52575    0    3 423740.000 4483.52

In [20]:
@show x_exp = Int.(round.(value.(expected_value_model[:x]).data))
@show y_exp = value.(expected_value_model[:y])
@show obj_exp = objective_value(expected_value_model)

## Comparing solutions
@show x_minmax = Int.(round.(value.(minimax_model[:x]).data))
@show y_minmax = value.(minimax_model[:y])
@show obj_minmax = objective_value(minimax_model)

x_exp = Int.(round.(value.(expected_value_model[:x]).data)) = [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
y_exp = value.(expected_value_model[:y]) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
   0.0
   0.0
 675.0
 344.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
obj_exp = objective_value(expected_value_model) = 4587.000000000005
x_minmax = Int.(round.(value.(minimax_model[:x]).data)) = [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
y_minmax = value.(minimax_model[:y]) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
   0.0
   0.0
 576.0
 443.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
obj_minmax = objective_value(minimax_model) = 5328.0


5328.0

In [21]:
## Column-and-constraint generation method
# Generates the main problem
function generate_main(instance)
    
    I, J, S, N, P, O, V, U, T, D, bigM = unroll_instance(instance)
    
    main = Model(myGurobi)
    set_silent(main)
    
    @variable(main, x[I], Bin)
    @variable(main, 0 <= y[I] <= bigM)
    @variable(main, θ)

    @constraint(main, sum(x[i] for i in I) <= N)
    
    @objective(main, Min, sum(O[i] * x[i] + V[i] * y[i] for i in I) + θ)
    
    return main  
end

# Adds the columns and constraints to the main problem
function add_columns_and_constraints(instance, main, d_bar, iter)   
    
    I, J, S, K, P, O, V, U, T, D, bigM = unroll_instance(instance)
    
    x = main[:x]
    y = main[:y]
    θ = main[:θ] 
    
    # Adding new variables to the main model. We are registering new variable sets at each iteration,
    # which requires the name of the variable to be passed as a Symbol.
    main[Symbol("w_$(iter)")] = @variable(main, [I,J], lower_bound = 0.0, base_name = "w_$(iter)")
    main[Symbol("z_$(iter)")] = @variable(main, [J], lower_bound = 0.0, base_name = "z_$(iter)")
    
    # For a cleaner code, we allocate to local variables 
    w = main[Symbol("w_$(iter)")]
    z = main[Symbol("z_$(iter)")]
    
    @constraint(main, 
        θ >= sum(T[i,j] * w[i,j] for i in I, j in J) + sum(U * z[j] for j in J) 
    )
    
    @constraint(main, [i in I],
        sum(w[i,j] for j in J) <=  y[i]
    )

    @constraint(main, [i in I], 
        sum(w[i,j] for j in J) <= x[i] * bigM
    )
    
    @constraint(main, [j in J],
        sum(w[i,j] for i in I) >= d_bar[j] - z[j]
    )
      
    return main
end  

# Solve the main problem
function solve_main(main)
    optimize!(main)
    return value.(main[:x]), value.(main[:y]), value(main[:θ]), objective_value(main)    
end

solve_main (generic function with 1 method)

In [22]:
# Define the CC subproblem. A couple of variants
function generate_and_solve_dualized_subproblem(instance, x_bar, y_bar, Γ)
    
    I, J, S, N, P, O, V, U, T, D, bigM, D_average, D_deviation = unroll_instance(instance)
     
    sub_dual = Model(myGurobi)
    set_optimizer_attribute(sub_dual, "NonConvex", 2) # Call spatial BB solver
 
    set_silent(sub_dual)
     
    @variable(sub_dual, u[I] <= 0)
    @variable(sub_dual, q[I] <= 0)
    @variable(sub_dual, r[J] >= 0)
    @variable(sub_dual, 0 <= g[J] <= 1)
    @variable(sub_dual, d[J] >= 0)
 
    @constraint(sub_dual, [i in I, j in J], 
       u[i] + q[i] + r[j] <= T[i,j]
    )
    @constraint(sub_dual, [j in J], 
       r[j] <= U
    )
 
    # Uncertainty set
    @constraint(sub_dual, [j in J],
       d[j] == D_average[j] + g[j] * D_deviation[j]
    )
 
    @constraint(sub_dual, [j in J],
       sum(g[j] for j in J) <= Γ
    )
 
    @objective(sub_dual, Max,  
       sum(bigM * x_bar[i] * q[i] for i in I) +
       sum(y_bar[i] * u[i] for i in I) +
       sum(d[j] * r[j] for j in J) 
    )   
    
    optimize!(sub_dual)
     
    d_bar = value.(sub_dual[:d])                     
    opt_value = objective_value(sub_dual)
     
    return d_bar, opt_value
 end
 
 
 # Uses the result that g is always integer anyways and apply linearisation
 function generate_and_solve_linearized_subproblem(instance, x_bar, y_bar, Γ)
     
     I, J, S, N, P, O, V, U, T, D, bigM, D_average, D_deviation = unroll_instance(instance)
      
     sub_dual = Model(myGurobi) 
     set_silent(sub_dual)
      
     #%
     @variable(sub_dual, u[I] <= 0)
     @variable(sub_dual, q[I] <= 0)
     @variable(sub_dual, r[J] >= 0)
     @variable(sub_dual, g[J], Bin)
    #  @variable(sub_dual, d[J] >= 0)
     @variable(sub_dual, b[J] >= 0)
  
     @constraint(sub_dual, [i in I, j in J], 
        u[i] + q[i] + r[j] <= T[i,j]
     )
     @constraint(sub_dual, [j in J], 
        r[j] <= U
     )
  
     # Uncertainty set
     #  @constraint(sub_dual, [j in J],
     #     d[j] == D_average[j] + g[j] * D_deviation
     #  )
  
     @constraint(sub_dual, [j in J],
        sum(g[j] for j in J) <= Γ
     )
 
     # Linearisation constraints
     @constraint(sub_dual, [j in J],
        b[j] <= r[j]
     )
  
     @constraint(sub_dual, [j in J],
        b[j] <= U * g[j]
     )
 
     @objective(sub_dual, Max,  
        sum(bigM * x_bar[i] * q[i] for i in I) +
        sum(y_bar[i] * u[i] for i in I) +
        sum(D_average[j] * r[j] + D_deviation[j] * b[j] for j in J) 
     )   
     
     optimize!(sub_dual)
      
     d_bar = [D_average[j] + value(g[j]) * D_deviation[j] for j in J]                 
     opt_value = objective_value(sub_dual)
      
     return d_bar, opt_value
  end
 
 # Bilevel JuMP version
 function generate_and_solve_bilevel_subproblem(instance, x_bar, y_bar, Γ)
     
    I, J, S, K, P, O, V, U, T, D, bigM, D_average, D_deviation = unroll_instance(instance)
      
    sub_dual = BilevelModel(myGurobi, mode = BilevelJuMP.SOS1Mode()) 
    set_silent(sub_dual)
      
    @variable(Upper(sub_dual), d[J] >= 0)
    @variable(Upper(sub_dual), 0 <= g[J] <= 1)
    @variable(Lower(sub_dual), w[I,J] >= 0)
    @variable(Lower(sub_dual), z[J] >= 0)
  
    @constraint(Lower(sub_dual), capBal[i in I],
       sum(w[i,j] for j in J) <=  y_bar[i]
    )
 
    @constraint(Lower(sub_dual), capLoc[i in I], 
       sum(w[i,j] for j in J) <= x_bar[i] * bigM
    )
    
    @constraint(Lower(sub_dual), demBal[j in J],
       sum(w[i,j] for i in I) >= d[j] - z[j]
    )
 
    # Uncertainty set
    @constraint(Upper(sub_dual), [j in J],
       d[j] == D_average[j] + g[j] * D_deviation[j]
    )
 
    @constraint(Upper(sub_dual), [j in J],
       sum(g[j] for j in J) <= Γ
    )
   
    @objective(Lower(sub_dual), Min, 
       sum(T[i,j] * w[i,j] for i in I, j in J) + sum(U * z[j] for j in J)
    ) 
 
    @objective(Upper(sub_dual), Max, 
       sum(T[i,j] * w[i,j] for i in I, j in J) + sum(U * z[j] for j in J)
    ) 
    
    optimize!(sub_dual)
    
    d_bar = [value(d[j]) for j in J]                 
    opt_value = objective_value(sub_dual)
    
    return d_bar, opt_value
  end

generate_and_solve_bilevel_subproblem (generic function with 1 method)

In [23]:
function cc_decomposition(ins; max_iter = 10, Γ = 10, sub_method = :linear)
    k = 1
    ϵ = 1e-4
    LB = -Inf
    UB = +Inf
    gap = +Inf
    x_bar = zeros(length(ins.I))
    y_bar = zeros(length(ins.J))
    
    println("\nStarting CCG decomposition...")
    start = time()    
    
    if sub_method == :linear
        d_bar, f_sub = generate_and_solve_linearized_subproblem(ins, x_bar, y_bar, Γ);
    elseif sub_method == :dual    
        d_bar, f_sub = generate_and_solve_dualized_subproblem(ins, x_bar, y_bar, Γ);
    elseif sub_method == :bilevel
        d_bar, f_sub = generate_and_solve_bilevel_subproblem(ins, x_bar, y_bar, Γ);
    else    
        error("Invalid subproblem solution method chosen.")
    end

    main = generate_main(ins)
    main = add_columns_and_constraints(ins, main, d_bar, k) 

    while k <= max_iter && gap > ϵ
        x_bar, y_bar, θ_bar, f_main = solve_main(main);
        
        if sub_method == :linear
            d_bar, f_sub = generate_and_solve_linearized_subproblem(ins, x_bar, y_bar, Γ);
        elseif sub_method == :dual    
            d_bar, f_sub = generate_and_solve_dualized_subproblem(ins, x_bar, y_bar, Γ);
        else # if sub_method == :bilevel
            d_bar, f_sub = generate_and_solve_bilevel_subproblem(ins, x_bar, y_bar, Γ);
        end
        
        LB = f_main
        UB = min(UB, f_main - θ_bar + f_sub)
        gap = abs((UB - LB) / UB)
        println("Iter $(k): UB: $(round(UB, digits=2)), LB: $(round(LB, digits=2)), gap: $(round(100*gap, digits=2))%")
        
        if gap <= ϵ
            stop = time()
            println("\nOptimal found.\n Objective value: $(round(UB, digits=2))\n Total time: $(round(stop-start, digits=2))s\n gap: $(round(100*gap, digits=2))%")
            return x_bar, y_bar, UB
        else    
            main = add_columns_and_constraints(ins, main, d_bar, k)
            k += 1
        end
    end
    println("Maximum number of iterations exceeded.")
end

cc_decomposition (generic function with 1 method)

In [24]:
@time x_bar, y_bar, UB = cc_decomposition(instance, max_iter = 10, Γ = 5, sub_method = :linear)
# @time x_bar, y_bar, UB = cc_decomposition(instance, max_iter = 10, Γ = 5, sub_method = :dual)
@time x_bar, y_bar, UB = cc_decomposition(instance, max_iter = 10, Γ = 5, sub_method = :bilevel)


Starting CCG decomposition...


Min 53 x[1] + 6 y[1] + 62 x[2] + 8 y[2] + 48 x[3] + y[3] + 76 x[4] + 2 y[4] + 54 x[5] + 5 y[5] + 56 x[6] + 7 y[6] + 79 x[7] + 10 y[7] + 72 x[8] + 7 y[8] + 60 x[9] + 10 y[9] + 70 x[10] + 8 y[10] + θ
Subject to
 

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 5.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0
 y[5] ≥ 0.0
 y[6] ≥ 0.0
 y[7] ≥ 0.0
 y[8] ≥ 0.0
 y[9] ≥ 0.0
 y[10] ≥ 0.0
 y[1] ≤ 1248.0
 y[2] ≤ 1248.0
 y[3] ≤ 1248.0
 y[4] ≤ 1248.0
 y[5] ≤ 1248.0
 y[6] ≤ 1248.0
 y[7] ≤ 1248.0
 y[8] ≤ 1248.0
 y[9] ≤ 1248.0
 y[10] ≤ 1248.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 x[10] binary

Iter 1: UB: 5084.0, LB: 4822.0, gap: 5.15%


Iter 2: UB: 5084.0, LB: 5046.64, gap: 0.73%


Iter 3: UB: 5082.77, LB: 5046.77, gap: 0.71%

...

Iter 1: UB: 5084.0, LB: 4842.0, gap: 4.76%


Iter 2: UB: 5084.0, LB: 5046.68, gap: 0.73%


Iter 3: UB: 5082.77, LB: 5046.77, gap: 0.71%


Iter 4: UB: 5064.69, LB: 5064.69, gap: 0.0%

Optimal found.
 Objective value: 5064.69
 Total time: 5.51s
 gap: 0.0%
  5.513324 seconds (6.33 M allocations: 674.186 MiB, 3.33% gc time, 70.43% compilation time)


(1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
 0.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0, 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{Float64}:
   0.0
   0.0
 486.6882591093122
 366.0
  92.99999999999955
   0.0
   0.0
   0.0
   0.0
   0.0, 5064.688259109311)

In [25]:
UB_k = []
for i in 1:10
    ⋅, ⋅, UB = cc_decomposition(instance, max_iter = 5, Γ = i,)
    push!(UB_k, UB)
end

plot(UB_k, 
    label = "obj. value",
    xlabel = "Γ")


Starting CCG decomposition...
Min 53 x[1] + 6 y[1] + 62 x[2] + 8 y[2] + 48 x[3] + y[3] + 76 x[4] + 2 y[4] + 54 x[5] + 5 y[5] + 56 x[6] + 7 y[6] + 79 x[7] + 10 y[7] + 72 x[8] + 7 y[8] + 60 x[9] + 10 y[9] + 70 x[10] + 8 y[10] + θ
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 5.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0
 y[5] ≥ 0.0
 y[6] ≥ 0.0
 y[7] ≥ 0.0
 y[8] ≥ 0.0
 y[9] ≥ 0.0
 y[10] ≥ 0.0
 y[1] ≤ 1248.0
 y[2] ≤ 1248.0
 y[3] ≤ 1248.0
 y[4] ≤ 1248.0
 y[5] ≤ 1248.0
 y[6] ≤ 1248.0
 y[7] ≤ 1248.0
 y[8] ≤ 1248.0
 y[9] ≤ 1248.0
 y[10] ≤ 1248.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 x[10] binary

Iter 1: UB: 4526.0, LB: 4416.0, gap: 2.43%
Iter 2: UB: 4519.82, LB: 4519.82, gap: 0.0%

Optimal found.
 Objective value: 4519.82
 Total time: 0.04s
 gap: 0.0%

Starting CCG decomposition...
Min 53 x[1] + 6 y[1] + 62 x[2] + 8 y[2] + 48 x[3] + y[3] + 76 x[4] + 2 y[4] + 54 x[5] + 5 

Iter 2: UB: 4680.0, LB: 4656.73, gap: 0.5%
Iter 3: UB: 4662.71, LB: 4662.71, gap: 0.0%

Optimal found.
 Objective value: 4662.71
 Total time: 0.15s
 gap: 0.0%

Starting CCG decomposition...
Min 53 x[1] + 6 y[1] + 62 x[2] + 8 y[2] + 48 x[3] + y[3] + 76 x[4] + 2 y[4] + 54 x[5] + 5 y[5] + 56 x[6] + 7 y[6] + 79 x[7] + 10 y[7] + 72 x[8] + 7 y[8] + 60 x[9] + 10 y[9] + 70 x[10] + 8 y[10] + θ
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 5.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0
 y[5] ≥ 0.0
 y[6] ≥ 0.0
 y[7] ≥ 0.0
 y[8] ≥ 0.0
 y[9] ≥ 0.0
 y[10] ≥ 0.0
 y[1] ≤ 1248.0
 y[2] ≤ 1248.0
 y[3] ≤ 1248.0
 y[4] ≤ 1248.0
 y[5] ≤ 1248.0
 y[6] ≤ 1248.0
 y[7] ≤ 1248.0
 y[8] ≤ 1248.0
 y[9] ≤ 1248.0
 y[10] ≤ 1248.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 x[10] binary

Iter 1: UB: 4824.0, LB: 4596.0, gap: 4.73%


Iter 2: UB: 4824.0, LB: 4803.64, gap: 0.42%


Iter 3: UB: 4803.64, LB: 4803.64, gap: 0.0%



Optimal found.
 Objective value: 4803.64
 Total time: 0.38s
 gap: 0.0%

Starting CCG decomposition...
Min 53 x[1] + 6 y[1] + 62 x[2] + 8 y[2] + 48 x[3] + y[3] + 76 x[4] + 2 y[4] + 54 x[5] + 5 y[5] + 56 x[6] + 7 y[6] + 79 x[7] + 10 y[7] + 72 x[8] + 7 y[8] + 60 x[9] + 10 y[9] + 70 x[10] + 8 y[10] + θ
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 5.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0
 y[5] ≥ 0.0
 y[6] ≥ 0.0
 y[7] ≥ 0.0
 y[8] ≥ 0.0
 y[9] ≥ 0.0
 y[10] ≥ 0.0
 y[1] ≤ 1248.0
 y[2] ≤ 1248.0
 y[3] ≤ 1248.0
 y[4] ≤ 1248.0
 y[5] ≤ 1248.0
 y[6] ≤ 1248.0
 y[7] ≤ 1248.0
 y[8] ≤ 1248.0
 y[9] ≤ 1248.0
 y[10] ≤ 1248.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 x[10] binary



...


Iter 2: UB: 5321.0, LB: 5305.57, gap: 0.29%


Iter 3: UB: 5308.7, LB: 5305.7, gap: 0.06%


Iter 4: UB: 5305.7, LB: 5305.7, gap: 0.0%



Optimal found.
 Objective value: 5305.7
 Total time: 34.67s
 gap: 0.0%

Starting CCG decomposition...
Min 53 x[1] + 6 y[1] + 62 x[2] + 8 y[2] + 48 x[3] + y[3] + 76 x[4] + 2 y[4] + 54 x[5] + 5 y[5] + 56 x[6] + 7 y[6] + 79 x[7] + 10 y[7] + 72 x[8] + 7 y[8] + 60 x[9] + 10 y[9] + 70 x[10] + 8 y[10] + θ
Subject to
 x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] ≤ 5.0
 y[1] ≥ 0.0
 y[2] ≥ 0.0
 y[3] ≥ 0.0
 y[4] ≥ 0.0
 y[5] ≥ 0.0
 y[6] ≥ 0.0
 y[7] ≥ 0.0
 y[8] ≥ 0.0
 y[9] ≥ 0.0
 y[10] ≥ 0.0
 y[1] ≤ 1248.0
 y[2] ≤ 1248.0
 y[3] ≤ 1248.0
 y[4] ≤ 1248.0
 y[5] ≤ 1248.0
 y[6] ≤ 1248.0
 y[7] ≤ 1248.0
 y[8] ≤ 1248.0
 y[9] ≤ 1248.0
 y[10] ≤ 1248.0
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary
 x[6] binary
 x[7] binary
 x[8] binary
 x[9] binary
 x[10] binary



Iter 1: UB: 5425.0, LB: 5256.0, gap: 3.12%
